In [2]:
import pandas as pd
import psycopg2
import config
import workdays
import datetime as dt


holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')
dt = dt.date.today()
vcto_0 = dt.strftime('%d/%m/%Y')
dt_pos = workdays.workday(dt, -1, holidays_br)
dt_1 = workdays.workday(dt, -1, holidays_b3)



df=pd.read_excel('bol.xlsx')


df['qtde']=df.apply(lambda row: row['qtde'] if row['ponta']=='C' else -row['qtde'],axis=1)


# def build_serie(x:str):
#     return x + ' BZ EQUITY'
# df['str_serie']=df['cod'].apply(lambda x: build_serie(x))

def get_equities_info(str_serie, d=dt.today()):
	data = d.strftime("%Y-%m-%d")
	query = "SELECT asst FROM b3up2data.equities_equityinstrumentfile \
		where tckrsymb='{}' AND rptdt='{}' ORDER BY rptdt desc"
	query = query.format(str_serie, data)
	db_conn = psycopg2.connect("host='PGKPTL01' dbname='Risk_DB' user='k_geral' password='Kapitalo@01'")
	with db_conn.cursor() as cursor:
		cursor.execute(query.format(data))
		try:
			ret = cursor.fetchall()[0][0]
		except Exception as err:
			raise ValueError(f'{str_serie} not found.')
		db_conn.close()
	return ret


def get_code(x):
    if "IND" in x:
        return "IND"
    elif "WIN" in x:
        return "WIN"
    else:
        return get_equities_info(x)


def get_type(x):
    if "IND" in x:
        return "Futuro"
    elif "WIN" in x:
        return "Futuro"
    else:
        return "Acao"

def get_serie(x):
    if "IND" in x:
        return x.replace("IND", "")
    elif "WIN" in x:
        return x.replace("WIN","")
    else:
        return x + " BZ EQUITY"




df['ALOCACAO'] = 'KAPITALO KAPPA MASTER FIM'
df['MESA'] = 'Kapitalo 11.1'
df['CLEARING'] = 'BRADESCO'
df['CONTRA'] = 'XP'
df['ESTRATEGIA'] = 'CashCarry'
df["codigo"] = df["cod"].apply(lambda x: get_code(x))
df["str_serie"] = df['cod'].apply(lambda x: get_serie(x))


df["TIPO"] = df["codigo"].apply(lambda x: get_type(x))


df=df[["ALOCACAO","MESA","ESTRATEGIA","CLEARING","CONTRA","TIPO", "codigo","str_serie","qtde","price"]]

df.to_excel('boleta.xlsx')

df.to_clipboard(excel=True)
